In [202]:
from __future__ import annotations

from typing import IO

import numpy as np

In [203]:
def single_bingo_array(f: IO) -> np.ndarray[int]:
    return np.array(
        [[int(x) for x in next(f).strip().split()] for _ in range(5)]
    )

Parse input file

In [204]:
with open("../input.txt") as f:
    
    # Read in the numbers
    numbers = [int(x) for x in f.readline().strip().split(",")]
    
    # Check for an empty line separating the boards.
    # If it's there read in a board, otherwise, we're done
    boards = []
    while f.readline():
        
        # Grab the next 5 lines to make a board
        boards.append(single_bingo_array(f))

Stack all boards into a high dimensional array

In [205]:
all_boards = np.dstack(boards)

Make a mask of what's been called in all the boards; i.e. an all false mask to start

In [206]:
called_boards = np.zeros(all_boards.shape, dtype=bool)

Iterate through the numbers and update the mask as we go. If we find a bingo stop and grab the winning board.

In [207]:
for num in numbers:
    
    # Update the called boards with the new number
    called_boards += (all_boards == num)
    
    # Sum the rows and columns to see if we have a bingo
    column_scores = called_boards.sum(axis=0)
    row_scores = called_boards.sum(axis=1)
    
    # Looking for a score of 5, which means bingo
    column_bingo = np.argwhere(column_scores == 5)
    row_bingo = np.argwhere(row_scores == 5)
    
    # If we actually found one
    if (column_winner := column_bingo.size > 0) or (row_winner := row_bingo.size > 0):
        if column_winner:
            assert column_bingo.shape[0] == 1, "expected 1 column winner"
            assert not row_winner, "expected just column winner"
            winning_board_num = column_bingo[0][1]
        
        if row_winner:
            assert row_bingo.shape[0] == 1, "expected 1 row winner"
            assert not column_winner, "expected just row winner"
            winning_board_num = row_bingo[0][1]
        
        break

Using the winning board mask we can figure out the sum of the uncalled numbers.

In [208]:
uncalled_sum = boards[winning_board_num][
    ~called_boards[:, :, winning_board_num]
].sum()

And lastly we multiply that sum by the last called number

In [209]:
uncalled_sum * num

38594